## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)


In [4]:
# 將分類的項目以one hot encoding來表達
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [5]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),
                             padding='same',input_shape = (32,32,3),activation='relu'))
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),
                             padding='same',input_shape = (32,32,3),activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 12s 239us/step - loss: 1.3704 - accuracy: 0.5185
Epoch 2/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.9561 - accuracy: 0.6614
Epoch 3/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.7865 - accuracy: 0.7232
Epoch 4/100
50000/50000 [==============================] - 5s 110us/step - loss: 0.6527 - accuracy: 0.7722
Epoch 5/100
50000/50000 [==============================] - 5s 110us/step - loss: 0.5284 - accuracy: 0.8141
Epoch 6/100
50000/50000 [==============================] - 6s 110us/step - loss: 0.4181 - accuracy: 0.8540
Epoch 7/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.3184 - accuracy: 0.8895
Epoch 8/100
50000/50000 [==============================] - 6s 110us/step - loss: 0.2390 - accuracy: 0.9177
Epoch 9/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.1757 - accuracy: 0.9388
Epoch 10/100
50000/50000 [==========

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.82777238e-16, 0.00000000e+00, 8.84503549e-28, 1.00000000e+00,
        9.88547243e-13, 0.00000000e+00, 1.50981553e-22, 0.00000000e+00,
        0.00000000e+00, 1.15494515e-33]], dtype=float32)